In [19]:
import requests
from pymongo import MongoClient
from pymongo import GEOSPHERE
import pandas as pd
import json
import os
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from getpass import getpass
import time
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
from collections import defaultdict
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

### MongoDB query ###

In [20]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
companies = db.get_collection("companies")

In [21]:
query = {"category_code": "games_video", "number_of_employees": {"$gte": 100, "$lte": 300}, "offices.country_code": {"$exists": True}, "offices.latitude": {"$exists": True}, "offices.latitude": {"$exists": True}}
projection = {"_id":0, "name":1, "category_code": 1, "offices.latitude": 1, "offices.longitude": 1, "number_of_employees": 1}
results = list(companies.find(query, projection))
results [:5]

[{'name': 'Livestream',
  'category_code': 'games_video',
  'number_of_employees': 120,
  'offices': [{'latitude': 40.726155, 'longitude': -73.995625}]},
 {'name': 'Ustream',
  'category_code': 'games_video',
  'number_of_employees': 250,
  'offices': [{'latitude': 37.392936, 'longitude': -122.07948},
   {'latitude': None, 'longitude': None},
   {'latitude': None, 'longitude': None}]},
 {'name': 'RockYou',
  'category_code': 'games_video',
  'number_of_employees': 106,
  'offices': [{'latitude': 37.484619, 'longitude': -122.206893}]},
 {'name': 'hi5',
  'category_code': 'games_video',
  'number_of_employees': 100,
  'offices': [{'latitude': 37.788668, 'longitude': -122.400558}]},
 {'name': 'Zango',
  'category_code': 'games_video',
  'number_of_employees': 225,
  'offices': [{'latitude': 47.579208, 'longitude': -122.155668}]}]

In [22]:
# checking how many game companies, with the number of employes from 100 to 300, per country

query = {"category_code": "games_video", "number_of_employees": {"$gte": 100, "$lte": 300}, "offices.country_code": {"$exists": True}, "offices.latitude": {"$exists": True}, "offices.latitude": {"$exists": True}}
projection = {"_id": 0, "offices.country_code": 1}

results = list(companies.find(query, projection))
country_count = defaultdict(int)

for result in results:
    country_code = result.get("offices", [{}])[0].get("country_code", "Unknown")
    country_count[country_code] += 1

for country, count in country_count.items():
    print(f"{country} - {count}")

USA - 21
SGP - 1
GBR - 2
IND - 2
ITA - 1
NLD - 1
CHE - 1
DEU - 1
SWE - 1
FRA - 1
UKR - 1


In [23]:
# As USA is the country with more game companies, checkong now the cities

query = {
    "category_code": "games_video",
    "number_of_employees": {"$gte": 100, "$lte": 300},
    "offices.country_code": "USA",
    "offices.city": {"$exists": True},
    "offices.latitude": {"$exists": True},
    "offices.longitude": {"$exists": True}
}

projection = {
    "_id": 0,
    "offices.city": 1
}

results = list(companies.find(query, projection))
us_city_count = defaultdict(int)

for result in results:
    city = result.get("offices", [{}])[0].get("city", "Unknown")
    us_city_count[city] += 1

print("City, Count")
for city, count in us_city_count.items():
    print(f"{city} - {count}")

City, Count
New York - 3
San Francisco - 7
Redwood City - 1
Bellevue - 1
Los Angeles - 3
Dundee - 1
Westwood - 1
Brooklyn - 1
Torino - 1
Sunnyvale - 3
Seattle - 1
Oxford - 1
Kharkov - 1


In [25]:
# looking for game companies, with number of employees Between 100 and 300

my_map = folium.Map(location=[40.726155, -121.997355], zoom_start=3)

for index, row in df.iterrows():
    if row['category_code'] == 'games_video':
        marker_color = 'red'
        marker_size = 'large'

    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],
        icon=folium.Icon(color=marker_color, icon_color='white', icon=marker_size)
    ).add_to(my_map)
my_map

In [26]:
query = {"category_code": "games_video", "number_of_employees": {"$gte": 100, "$lte": 300}, "offices.country_code": {"$exists": True}, "offices.latitude": {"$exists": True}, "offices.latitude": {"$exists": True}}
projection = {"_id":0, "name":1, "category_code": 1, "offices.latitude": 1, "offices.longitude": 1, "number_of_employees": 1}
results2 = list(companies.find(query, projection))
results2 [:5]

[{'name': 'Livestream',
  'category_code': 'games_video',
  'number_of_employees': 120,
  'offices': [{'latitude': 40.726155, 'longitude': -73.995625}]},
 {'name': 'Ustream',
  'category_code': 'games_video',
  'number_of_employees': 250,
  'offices': [{'latitude': 37.392936, 'longitude': -122.07948},
   {'latitude': None, 'longitude': None},
   {'latitude': None, 'longitude': None}]},
 {'name': 'RockYou',
  'category_code': 'games_video',
  'number_of_employees': 106,
  'offices': [{'latitude': 37.484619, 'longitude': -122.206893}]},
 {'name': 'hi5',
  'category_code': 'games_video',
  'number_of_employees': 100,
  'offices': [{'latitude': 37.788668, 'longitude': -122.400558}]},
 {'name': 'Zango',
  'category_code': 'games_video',
  'number_of_employees': 225,
  'offices': [{'latitude': 47.579208, 'longitude': -122.155668}]}]

In [27]:
df = pd.DataFrame(results2)
df

name category_code  number_of_employees  \
0              Livestream   games_video                  120   
1                 Ustream   games_video                  250   
2                 RockYou   games_video                  106   
3                     hi5   games_video                  100   
4                   Zango   games_video                  225   
5                    hulu   games_video                  153   
6                   mig33   games_video                  100   
7                   Zynga   games_video                  115   
8               GameDuell   games_video                  200   
9   Social Gaming Network   games_video                  100   
10                 Boonty   games_video                  150   
11        Realtime Worlds   games_video                  225   
12            Modern Feed   games_video                  123   
13                Turbine   games_video                  200   
14                   iWin   games_video                  103   
15                 VBS TV   games_video                  100   
16                 ngmoco   games_video                  150   
17          Whereismyboss   games_video                  100   
18               deltatre   games_video                  225   
19             Indiagames   games_video                  300   
20             SPIL GAMES   games_video                  300   
21               Miniclip   games_video                  140   
22           RGB Networks   games_video                  125   
23            i-Jet Media   games_video                  150   
24   Turtle Entertainment   games_video                  175   
25     Unity Technologies   games_video                  200   
26                 Z2Live   games_video                  130   
27              ZillionTV   games_video                  100   
28          NaturalMotion   games_video                  150   
29                 Aspiro   games_video                  115   
30              2 Minutes   games_video                  105   
31                  Exent   games_video                  100   
32       G5 Entertainment   games_video                  107   

                                              offices  
0   [{'latitude': 40.726155, 'longitude': -73.9956...  
1   [{'latitude': 37.392936, 'longitude': -122.079...  
2   [{'latitude': 37.484619, 'longitude': -122.206...  
3   [{'latitude': 37.788668, 'longitude': -122.400...  
4   [{'latitude': 47.579208, 'longitude': -122.155...  
5   [{'latitude': 34.031276, 'longitude': -118.456...  
6   [{'latitude': 37.580304, 'longitude': -122.343...  
7   [{'latitude': 37.765158, 'longitude': -122.404...  
8   [{'latitude': None, 'longitude': None}, {'lati...  
9   [{'latitude': 37.446823, 'longitude': -122.161...  
10  [{'latitude': 40.717248, 'longitude': -74.0026...  
11  [{'latitude': 56.461428, 'longitude': -2.96811...  
12  [{'latitude': 34.085749, 'longitude': -118.343...  
13  [{'latitude': 42.214483, 'longitude': -71.1859...  
14  [{'latitude': 37.781689, 'longitude': -122.391...  
15  [{'latitude': 40.720764, 'longitude': -73.9579...  
16  [{'latitude': 37.7705096, 'longitude': -122.40...  
17            [{'latitude': None, 'longitude': None}]  
18  [{'latitude': 45.058576, 'longitude': 7.649614...  
19            [{'latitude': None, 'longitude': None}]  
20  [{'latitude': 52.220726, 'longitude': 5.198824...  
21            [{'latitude': None, 'longitude': None}]  
22  [{'latitude': 37.4123706, 'longitude': -122.02...  
23  [{'latitude': 37.384239, 'longitude': -122.012...  
24  [{'latitude': 50.9272941, 'longitude': 6.97649...  
25  [{'latitude': 55.6924605, 'longitude': 12.5301...  
26  [{'latitude': 47.6103008, 'longitude': -122.33...  
27  [{'latitude': 37.3738725, 'longitude': -121.99...  
28  [{'latitude': None, 'longitude': None}, {'lati...  
29            [{'latitude': None, 'longitude': None}]  
30  [{'latitude': 48.8502056, 'longitude': 2.36980...  
31  [{'latitude': 40.7523796, 'longitude': -74.005..

In [28]:
df['latitude'] = df['offices'].apply(lambda x: x[0]['latitude'])
df['longitude'] = df['offices'].apply(lambda x: x[0]['longitude'])
df

name category_code  number_of_employees  \
0              Livestream   games_video                  120   
1                 Ustream   games_video                  250   
2                 RockYou   games_video                  106   
3                     hi5   games_video                  100   
4                   Zango   games_video                  225   
5                    hulu   games_video                  153   
6                   mig33   games_video                  100   
7                   Zynga   games_video                  115   
8               GameDuell   games_video                  200   
9   Social Gaming Network   games_video                  100   
10                 Boonty   games_video                  150   
11        Realtime Worlds   games_video                  225   
12            Modern Feed   games_video                  123   
13                Turbine   games_video                  200   
14                   iWin   games_video                  103   
15                 VBS TV   games_video                  100   
16                 ngmoco   games_video                  150   
17          Whereismyboss   games_video                  100   
18               deltatre   games_video                  225   
19             Indiagames   games_video                  300   
20             SPIL GAMES   games_video                  300   
21               Miniclip   games_video                  140   
22           RGB Networks   games_video                  125   
23            i-Jet Media   games_video                  150   
24   Turtle Entertainment   games_video                  175   
25     Unity Technologies   games_video                  200   
26                 Z2Live   games_video                  130   
27              ZillionTV   games_video                  100   
28          NaturalMotion   games_video                  150   
29                 Aspiro   games_video                  115   
30              2 Minutes   games_video                  105   
31                  Exent   games_video                  100   
32       G5 Entertainment   games_video                  107   

                                              offices   latitude   longitude  
0   [{'latitude': 40.726155, 'longitude': -73.9956...  40.726155  -73.995625  
1   [{'latitude': 37.392936, 'longitude': -122.079...  37.392936 -122.079480  
2   [{'latitude': 37.484619, 'longitude': -122.206...  37.484619 -122.206893  
3   [{'latitude': 37.788668, 'longitude': -122.400...  37.788668 -122.400558  
4   [{'latitude': 47.579208, 'longitude': -122.155...  47.579208 -122.155668  
5   [{'latitude': 34.031276, 'longitude': -118.456...  34.031276 -118.456806  
6   [{'latitude': 37.580304, 'longitude': -122.343...  37.580304 -122.343679  
7   [{'latitude': 37.765158, 'longitude': -122.404...  37.765158 -122.404234  
8   [{'latitude': None, 'longitude': None}, {'lati...        NaN         NaN  
9   [{'latitude': 37.446823, 'longitude': -122.161...  37.446823 -122.161523  
10  [{'latitude': 40.717248, 'longitude': -74.0026...  40.717248  -74.002662  
11  [{'latitude': 56.461428, 'longitude': -2.96811...  56.461428   -2.968111  
12  [{'latitude': 34.085749, 'longitude': -118.343...  34.085749 -118.343791  
13  [{'latitude': 42.214483, 'longitude': -71.1859...  42.214483  -71.185964  
14  [{'latitude': 37.781689, 'longitude': -122.391...  37.781689 -122.391061  
15  [{'latitude': 40.720764, 'longitude': -73.9579...  40.720764  -73.957968  
16  [{'latitude': 37.7705096, 'longitude': -122.40...  37.770510 -122.401581  
17            [{'latitude': None, 'longitude': None}]        NaN         NaN  
18  [{'latitude': 45.058576, 'longitude': 7.649614...  45.058576    7.649614  
19            [{'latitude': None, 'longitude': None}]        NaN         NaN  
20  [{'latitude': 52.220726, 'longitude': 5.198824...  52.220726    5.198824  
21            [{'latitude': None, 'longitude': None}]        NaN         NaN  
22  [{'latitude': 37.412370

In [16]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df

name category_code  number_of_employees  \
0              Livestream   games_video                  120   
1                 Ustream   games_video                  250   
2                 RockYou   games_video                  106   
3                     hi5   games_video                  100   
4                   Zango   games_video                  225   
5                    hulu   games_video                  153   
6                   mig33   games_video                  100   
7                   Zynga   games_video                  115   
8   Social Gaming Network   games_video                  100   
9                  Boonty   games_video                  150   
10        Realtime Worlds   games_video                  225   
11            Modern Feed   games_video                  123   
12                Turbine   games_video                  200   
13                   iWin   games_video                  103   
14                 VBS TV   games_video                  100   
15                 ngmoco   games_video                  150   
16               deltatre   games_video                  225   
17             SPIL GAMES   games_video                  300   
18           RGB Networks   games_video                  125   
19            i-Jet Media   games_video                  150   
20   Turtle Entertainment   games_video                  175   
21     Unity Technologies   games_video                  200   
22                 Z2Live   games_video                  130   
23              ZillionTV   games_video                  100   
24              2 Minutes   games_video                  105   
25                  Exent   games_video                  100   

                                              offices   latitude   longitude  
0   [{'latitude': 40.726155, 'longitude': -73.9956...  40.726155  -73.995625  
1   [{'latitude': 37.392936, 'longitude': -122.079...  37.392936 -122.079480  
2   [{'latitude': 37.484619, 'longitude': -122.206...  37.484619 -122.206893  
3   [{'latitude': 37.788668, 'longitude': -122.400...  37.788668 -122.400558  
4   [{'latitude': 47.579208, 'longitude': -122.155...  47.579208 -122.155668  
5   [{'latitude': 34.031276, 'longitude': -118.456...  34.031276 -118.456806  
6   [{'latitude': 37.580304, 'longitude': -122.343...  37.580304 -122.343679  
7   [{'latitude': 37.765158, 'longitude': -122.404...  37.765158 -122.404234  
8   [{'latitude': 37.446823, 'longitude': -122.161...  37.446823 -122.161523  
9   [{'latitude': 40.717248, 'longitude': -74.0026...  40.717248  -74.002662  
10  [{'latitude': 56.461428, 'longitude': -2.96811...  56.461428   -2.968111  
11  [{'latitude': 34.085749, 'longitude': -118.343...  34.085749 -118.343791  
12  [{'latitude': 42.214483, 'longitude': -71.1859...  42.214483  -71.185964  
13  [{'latitude': 37.781689, 'longitude': -122.391...  37.781689 -122.391061  
14  [{'latitude': 40.720764, 'longitude': -73.9579...  40.720764  -73.957968  
15  [{'latitude': 37.7705096, 'longitude': -122.40...  37.770510 -122.401581  
16  [{'latitude': 45.058576, 'longitude': 7.649614...  45.058576    7.649614  
17  [{'latitude': 52.220726, 'longitude': 5.198824...  52.220726    5.198824  
18  [{'latitude': 37.4123706, 'longitude': -122.02...  37.412371 -122.021627  
19  [{'latitude': 37.384239, 'longitude': -122.012...  37.384239 -122.012475  
20  [{'latitude': 50.9272941, 'longitude': 6.97649...  50.927294    6.976498  
21  [{'latitude': 55.6924605, 'longitude': 12.5301...  55.692461   12.530107  
22  [{'latitude': 47.6103008, 'longitude': -122.33...  47.610301 -122.339978  
23  [{'latitude': 37.3738725, 'longitude': -121.99...  37.373872 -121.997355  
24  [{'latitude': 48.8502056, 'longitude': 2.36980...  48.850206    2.369803  
25  [{'latitude': 40.7523796, 'longitude': -74.005...  40.752380  -74.005568

In [29]:
token = getpass()

········


In [18]:
client_id = getpass()

········


In [ ]:
client_secret = getpass()

### Getting nearby venues through Foursquare for San Fracisco ### 

In [30]:
where = "San Francisco, USA"

In [31]:
lat = 37.765158
lon = -122.404234

In [32]:
# locating a random company in San Francisco from the game and number of employees criteria

latitude = 37.765158
longitude = -122.404234

m = folium.Map(location=[latitude, longitude], zoom_start=12)
folium.Marker([latitude, longitude], popup="Your Location").add_to(m)
m

In [33]:
# checking for close Child School

def requests_for_foursquare (query, lat, lon, radius=2000, limit=1):
    lat = 37.765158
    lon = -122.404234 
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [34]:
child_school = requests_for_foursquare ("Preschool", lat, lon, radius=2000, limit=5)
child_school

{'results': [{'fsq_id': '5dbb78957e097c00089cb7c9',
   'categories': [{'id': 12056,
     'name': 'Preschool',
     'short_name': 'Preschool',
     'plural_name': 'Preschools',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'LikelyOpen',
   'distance': 1229,
   'geocodes': {'main': {'latitude': 37.761362, 'longitude': -122.41741},
    'roof': {'latitude': 37.761362, 'longitude': -122.41741}},
   'link': '/v3/places/5dbb78957e097c00089cb7c9',
   'location': {'address': '722 S Van Ness Ave',
    'census_block': '060750208022000',
    'country': 'US',
    'cross_street': '18th St',
    'dma': 'San Francisco-Oakland-San Jose',
    'formatted_address': '722 S Van Ness Ave (18th St), San Francisco, CA 94110',
    'locality': 'San Francisco',
    'postcode': '94110',
    'region': 'CA'},
   'name': 'Petite Explorer Preschool',
   'related_places': {},
   'timezone': 'America/Los_Angeles'},
  {'f

In [35]:
near_child_school = []
for venue in child_school["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    near_child_school.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

child_school_df = pd.DataFrame(near_child_school)

child_school_df

Name   Latitude   Longitude  Distance
0              Petite Explorer Preschool  37.761362 -122.417410      1229
1                   Apple Tree Preschool  37.777038 -122.405075      1313
2  Potrero Kids Daniel Webster Preschool  37.760587 -122.395863       893
3                      Studio Montessori  37.771043 -122.404865       658
4               Good Samaritan Preschool  37.751894 -122.406603      1488

In [36]:
# Creating a map centered around a location of child school 
m = folium.Map(location=[child_school_df.iloc[0]['Latitude'], child_school_df.iloc[0]['Longitude']], zoom_start=12)

for index, row in child_school_df.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], 
                  tooltip=row['Name'], 
                  icon=folium.Icon(color='green')).add_to(m)

company_location = [37.765158, -122.404234]
folium.Marker(company_location, 
              tooltip='Zynga',
              icon=folium.Icon(color='red', icon='cloud')).add_to(m)

m

In [37]:
# checking for close Starbucks

def requests_for_foursquare (query, lat, lon, radius=2000, limit=1):
    lat = 37.765158
    lon = -122.404234 
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [38]:
starbucks = requests_for_foursquare ("Starbucks", lat, lon, radius=1000, limit=5)
starbucks

{'results': [{'fsq_id': '4a55303af964a520d0b31fe3',
   'categories': [{'id': 13035,
     'name': 'Coffee Shop',
     'short_name': 'Coffee Shop',
     'plural_name': 'Coffee Shops',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'}}],
   'chains': [{'id': 'ab4c54c0-d68a-012e-5619-003048cad9da',
     'name': 'Starbucks'}],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 105,
   'geocodes': {'main': {'latitude': 37.765796, 'longitude': -122.403409},
    'roof': {'latitude': 37.765796, 'longitude': -122.403409}},
   'link': '/v3/places/4a55303af964a520d0b31fe3',
   'location': {'address': '350 Rhode Island St',
    'address_extended': 'Ste 120',
    'census_block': '060750607021028',
    'country': 'US',
    'cross_street': 'at 16th St',
    'dma': 'San Francisco-Oakland-San Jose',
    'formatted_address': '350 Rhode Island St (at 16th St), San Francisco, CA 94103',
    'locality': 'San Francisco',
    'postcode': '94103',
    

In [39]:
near_starbucks = []
for venue in starbucks["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    near_starbucks.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

starbucks_df = pd.DataFrame(near_starbucks)

starbucks_df

Name   Latitude   Longitude  Distance
0  Starbucks  37.765796 -122.403409       105
1  Starbucks  37.766862 -122.409564       493
2  Starbucks  37.767121 -122.393738       935
3  Starbucks  37.762898 -122.410549       600
4  Starbucks  37.770294 -122.403435       575

In [40]:
# Creating a map centered around a location of Starbucks
m = folium.Map(location=[starbucks_df.iloc[0]['Latitude'], starbucks_df.iloc[0]['Longitude']], zoom_start=12)

for index, row in starbucks_df.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], 
                  tooltip=row['Name'], 
                  icon=folium.Icon(color='green')).add_to(m)

company_location = [37.765158, -122.404234]
folium.Marker(company_location, 
              tooltip='Zynga',
              icon=folium.Icon(color='red', icon='cloud')).add_to(m)

m

In [41]:
# checking for close Vegan Restaurants

def requests_for_foursquare (query, lat, lon, radius=1000, limit=1):
    lat = 37.765158
    lon = -122.404234 
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [42]:
vegan_restaurant = requests_for_foursquare ("Vegan", lat, lon, radius=1000, limit=5)
vegan_restaurant

{'results': [{'fsq_id': '475fbd10f964a520de4c1fe3',
   'categories': [{'id': 17069,
     'name': 'Grocery Store',
     'short_name': 'Grocery Store',
     'plural_name': 'Grocery Stores',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
      'suffix': '.png'}}],
   'chains': [{'id': 'c356152c-f98c-4c43-8a1e-1d16cc9ea1c8',
     'name': 'Whole Foods'}],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 167,
   'geocodes': {'main': {'latitude': 37.764381, 'longitude': -122.40268},
    'roof': {'latitude': 37.764381, 'longitude': -122.40268}},
   'link': '/v3/places/475fbd10f964a520de4c1fe3',
   'location': {'address': '450 Rhode Island St',
    'census_block': '060750227041003',
    'country': 'US',
    'cross_street': 'at 17th St',
    'dma': 'San Francisco-Oakland-San Jose',
    'formatted_address': '450 Rhode Island St (at 17th St), San Francisco, CA 94107',
    'locality': 'San Francisco',
    'postcode': '94107',
    'region': 'CA'},
   'name'

In [43]:
vegan = []
for venue in vegan_restaurant["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    vegan.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

vegan_restaurant_df = pd.DataFrame(vegan)

vegan_restaurant_df

Name   Latitude   Longitude  Distance
0            Whole Foods  37.764381 -122.402680       167
1            Stable Cafe  37.763254 -122.415300       992
2  SOMA StrEat Food Park  37.769745 -122.412028       827
3        Goat Hill Pizza  37.762371 -122.397760       652
4             Atlas Cafe  37.758973 -122.411464       934

In [44]:
# Creating a map centered around a location of vegan restaurant
m = folium.Map(location=[vegan_restaurant_df.iloc[0]['Latitude'], vegan_restaurant_df.iloc[0]['Longitude']], zoom_start=13)

for index, row in vegan_restaurant_df.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], 
                  tooltip=row['Name'], 
                  icon=folium.Icon(color='green')).add_to(m)

company_location = [37.765158, -122.404234]
folium.Marker(company_location, 
              tooltip='Zynga',
              icon=folium.Icon(color='red', icon='cloud')).add_to(m)

m

In [45]:
# checking for close Basketball Stadium 


def requests_for_foursquare (query, lat, lon, radius=1000, limit=1):
    lat = 37.765158
    lon = -122.404234 
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [46]:
basketball_stadium = requests_for_foursquare ("basketball", lat, lon, radius=1000, limit=5)
basketball_stadium

{'results': [{'fsq_id': '4bf5e710004ed13a55a441a0',
   'categories': [{'id': 10000,
     'name': 'Arts and Entertainment',
     'short_name': 'Arts & Entertainment',
     'plural_name': 'Arts and Entertainment',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_',
      'suffix': '.png'}},
    {'id': 18008,
     'name': 'Basketball Court',
     'short_name': 'Basketball Court',
     'plural_name': 'Basketball Courts',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/basketballcourt_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 932,
   'geocodes': {'main': {'latitude': 37.77208, 'longitude': -122.397686}},
   'link': '/v3/places/4bf5e710004ed13a55a441a0',
   'location': {'address': 'Berry St',
    'census_block': '060750607033005',
    'country': 'US',
    'cross_street': 'I-280',
    'dma': 'San Francisco-Oakland-San Jose',
    'formatted_address': 'Berry St (I-2

In [47]:
basketball = []
for venue in basketball_stadium["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    basketball.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

basketball_stadium_df = pd.DataFrame(basketball)

basketball_stadium_df

Name   Latitude   Longitude  Distance
0  Berry Basketball Courts  37.772080 -122.397686       932
1               NBA Nation  37.759168 -122.412335       987
2       Jackson Playground  37.763975 -122.399001       420
3       Mission Creek Park  37.770963 -122.398624      1245
4    SOMA StrEat Food Park  37.769745 -122.412028       827

In [49]:
# Creating a map centered around a location of basketball stadium
m = folium.Map(location=[basketball_stadium_df.iloc[0]['Latitude'], basketball_stadium_df.iloc[0]['Longitude']], zoom_start=12)

for index, row in basketball_stadium_df.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], 
                  tooltip=row['Name'], 
                  icon=folium.Icon(color='green')).add_to(m)

company_location = [37.765158, -122.404234]
folium.Marker(company_location, 
              tooltip='Zynga',
              icon=folium.Icon(color='red', icon='cloud')).add_to(m)

m

In [52]:
# checking for close Pet Store 


def requests_for_foursquare (query, lat, lon, radius=1000, limit=1):
    lat = 37.765158
    lon = -122.404234 
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [53]:
pet_shop = requests_for_foursquare ("Pet Store", lat, lon, radius=1000, limit=5)
pet_shop

{'results': [{'fsq_id': '4a21b55df964a520327d1fe3',
   'categories': [{'id': 11134,
     'name': 'Pet Grooming Service',
     'short_name': 'Pet Grooming Service',
     'plural_name': 'Pet Grooming Services',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/dogrun_',
      'suffix': '.png'}},
    {'id': 17110,
     'name': 'Pet Supplies Store',
     'short_name': 'Pet Supplies Store',
     'plural_name': 'Pet Supplies Stores',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pet_store_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 862,
   'geocodes': {'drop_off': {'latitude': 37.76412, 'longitude': -122.394544},
    'main': {'latitude': 37.764155, 'longitude': -122.394496},
    'roof': {'latitude': 37.764155, 'longitude': -122.394496}},
   'link': '/v3/places/4a21b55df964a520327d1fe3',
   'location': {'address': '199 Mississippi St',
    'census_block': '060750227021001',
    'countr

In [54]:
pet_shop_ = []
for venue in pet_shop["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    pet_shop_.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

pet_shop_df = pd.DataFrame(pet_shop_)

pet_shop_df

Name   Latitude   Longitude  Distance
0  Pawtrero Hill BathHouse & Feed  37.764155 -122.394496       862
1                           Petco  37.766509 -122.410332       540
2                          George  37.761277 -122.410811       721
3               Bernie's Grooming  37.766882 -122.412719       796
4                   Mission: Cats  37.762355 -122.414243       922

In [55]:
# Creating a map centered around a location of Pet shop
m = folium.Map(location=[pet_shop_df.iloc[0]['Latitude'], pet_shop_df.iloc[0]['Longitude']], zoom_start=12)

for index, row in pet_shop_df.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], 
                  tooltip=row['Name'], 
                  icon=folium.Icon(color='green')).add_to(m)

company_location = [37.765158, -122.404234]
folium.Marker(company_location, 
              tooltip='Zynga',
              icon=folium.Icon(color='red', icon='cloud')).add_to(m)

m

In [56]:
# checking for close Airport 

def requests_for_foursquare (query, lat, lon, radius=500, limit=1):
    lat = 37.765158
    lon = -122.404234 
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [57]:
airport = requests_for_foursquare ("International Airport", lat, lon, radius=50000, limit=5)
airport

{'results': [{'fsq_id': '41059b00f964a520850b1fe3',
   'categories': [{'id': 19031,
     'name': 'Airport',
     'short_name': 'Airport',
     'plural_name': 'Airports',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/airport_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 16559,
   'geocodes': {'main': {'latitude': 37.61649, 'longitude': -122.38681}},
   'link': '/v3/places/41059b00f964a520850b1fe3',
   'location': {'address_extended': 'Ste 650',
    'census_block': '060819843001006',
    'country': 'US',
    'cross_street': 'at Link Rd',
    'dma': 'San Francisco-Oakland-San Jose',
    'formatted_address': 'at Link Rd, San Francisco, CA 94128',
    'locality': 'San Francisco',
    'po_box': 'P.O. Box 8097',
    'postcode': '94128',
    'region': 'CA'},
   'name': 'San Francisco International Airport (SFO)',
   'related_places': {'children': [{'fsq_id': '4db37642316a3bec52549b82',
      'categories': [{'id': 19

In [58]:
airport_ = []
for venue in airport["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    airport_.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

airport_df = pd.DataFrame(airport_)

airport_df

Name   Latitude   Longitude  Distance
0  San Francisco International Airport (SFO)  37.616490 -122.386810     16559
1        Oakland International Airport (OAK)  37.712364 -122.213326     17781

In [59]:
# Creating a map centered around a location of an Internation Airport

latitude = 37.765158
longitude = -122.404234

geolocator = Nominatim(user_agent="airport_finder")
airport_coordinates = (37.6213129, -122.3811443)  # San Francisco International Airport (SFO)
company_coordinates = (latitude, longitude)

m = folium.Map(location=company_coordinates, zoom_start=10)

folium.Marker(airport_coordinates, icon=folium.Icon(color='blue'), popup="San Francisco International Airport (SFO)").add_to(m)
folium.Marker(company_coordinates, icon=folium.Icon(color='green'), popup="Company Location").add_to(m)

m

In [60]:
# checking the location from the closest airport from San Fracisco - USA

latitude = 37.765158
longitude = -122.404234

geolocator = Nominatim(user_agent="airport_finder")
airport_coordinates = (37.6213129, -122.3811443)  # San Francisco International Airport (SFO)
distance = geodesic((latitude, longitude), airport_coordinates).miles
print(f"The closest airport is San Francisco International Airport (SFO), which is approximately {distance:.2f} miles away.")

The closest airport is San Francisco International Airport (SFO), which is approximately 10.00 miles away.


### Getting nearby venues through Foursquare for Los Angeles ### 

In [61]:
where_2 = "Los Angeles, USA"
latitude2 = 34.031276
longitude2 = -118.456806

In [62]:
# locating a random company in Los Angeles from the game and number of employees criteria
latitude2 = 34.031276
longitude2 = -118.456806

m = folium.Map(location=[latitude2, longitude2], zoom_start=10)

icon = folium.Icon(icon="cloud")
folium.Marker([latitude2, longitude2], popup="Your Location", icon=icon).add_to(m)

m

In [63]:
# checking for close Child School

def requests_for_foursquare (query, lat, lon, radius=1000, limit=1):
    lat = 34.031276
    lon = -118.456806 
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [66]:
child_school_la = requests_for_foursquare ("Preschool", lat, lon, radius=1000, limit=5)
child_school_la

{'results': [{'fsq_id': '8f15b629dbf247579e380e28',
   'categories': [{'id': 12056,
     'name': 'Preschool',
     'short_name': 'Preschool',
     'plural_name': 'Preschools',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'Unsure',
   'distance': 597,
   'geocodes': {'drop_off': {'latitude': 34.036799, 'longitude': -118.457263},
    'main': {'latitude': 34.036621, 'longitude': -118.457464},
    'roof': {'latitude': 34.036621, 'longitude': -118.457464}},
   'link': '/v3/places/8f15b629dbf247579e380e28',
   'location': {'address': '1815 Brockton Ave',
    'census_block': '060372676003002',
    'country': 'US',
    'dma': 'Los Angeles',
    'formatted_address': '1815 Brockton Ave, Los Angeles, CA 90025',
    'locality': 'Los Angeles',
    'postcode': '90025',
    'region': 'CA'},
   'name': 'Our World Preschool',
   'related_places': {}},
  {'fsq_id': '4bc3d115b492d13a1a5ca960',
   'catego

In [67]:
near_child_school_la = []
for venue in child_school_la["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    near_child_school_la.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

child_school_df2 = pd.DataFrame(near_child_school_la)

child_school_df2

Name   Latitude   Longitude  Distance
0            Our World Preschool  34.036621 -118.457464       597
1  Little Village Nursery School  34.030477 -118.448647       744
2             Mindful Heart Kids  34.036955 -118.454715       659
3                     Maple Tree  34.029973 -118.448566       772
4      Sunny Childcare Preschool  34.032045 -118.447335       876

In [69]:
# Creating a map centered around a location of child school 
m = folium.Map(location=[child_school_df2.iloc[0]['Latitude'], child_school_df2.iloc[0]['Longitude']], zoom_start=12)

for index, row in child_school_df2.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], 
                  tooltip=row['Name'], 
                  icon=folium.Icon(color='green')).add_to(m)

company_location = [34.031276, -118.456806]
folium.Marker(company_location, 
              tooltip='hulu',
              icon=folium.Icon(color='red', icon='cloud')).add_to(m)

m

In [70]:
# checking for close Starbucks

def requests_for_foursquare (query, lat, lon, radius=2000, limit=1):
    lat = 34.031276 
    lon = -118.456806 
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [71]:
starbucks_la = requests_for_foursquare ("Starbucks", lat, lon, radius=1000, limit=5)
starbucks_la

{'results': [{'fsq_id': '570eae75498e636a8f86a3c8',
   'categories': [{'id': 13035,
     'name': 'Coffee Shop',
     'short_name': 'Coffee Shop',
     'plural_name': 'Coffee Shops',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'}}],
   'chains': [{'id': 'ab4c54c0-d68a-012e-5619-003048cad9da',
     'name': 'Starbucks'}],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 859,
   'geocodes': {'main': {'latitude': 34.024394, 'longitude': -118.460891},
    'roof': {'latitude': 34.024394, 'longitude': -118.460891}},
   'link': '/v3/places/570eae75498e636a8f86a3c8',
   'location': {'address': '2727 Pico Blvd',
    'census_block': '060377018013009',
    'country': 'US',
    'cross_street': 'Stewart',
    'dma': 'Los Angeles',
    'formatted_address': '2727 Pico Blvd (Stewart), Santa Monica, CA 90405',
    'locality': 'Santa Monica',
    'postcode': '90405',
    'region': 'CA'},
   'name': 'Starbucks',
   'related_places': {},
   'ti

In [72]:
near_starbucks_la = []
for venue in starbucks_la["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    near_starbucks_la.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

starbucks_df_la = pd.DataFrame(near_starbucks_la)

starbucks_df_la

Name   Latitude   Longitude  Distance
0  Starbucks  34.024394 -118.460891       859
1  Starbucks  34.034989 -118.449162       812

In [73]:
# Creating a map centered around a location of Starbucks
m = folium.Map(location=[starbucks_df_la.iloc[0]['Latitude'], starbucks_df_la.iloc[0]['Longitude']], zoom_start=12)

for index, row in starbucks_df_la.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], 
                  tooltip=row['Name'], 
                  icon=folium.Icon(color='green')).add_to(m)

company_location = [34.031276, -118.456806]
folium.Marker(company_location, 
              tooltip='hulu',
              icon=folium.Icon(color='red', icon='cloud')).add_to(m)

m

In [74]:
# checking for close Vegan Restaurants

def requests_for_foursquare (query, lat, lon, radius=1000, limit=1):
    lat = 34.031276 
    lon = -118.456806  
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [75]:
vegan_restaurant_la = requests_for_foursquare ("Vegan", lat, lon, radius=1000, limit=5)
vegan_restaurant_la

{'results': [{'fsq_id': '56fea33a498e4c6f24e1bf4b',
   'categories': [{'id': 13028,
     'name': 'Breakfast Spot',
     'short_name': 'Breakfast',
     'plural_name': 'Breakfast Spots',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/breakfast_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'Unsure',
   'distance': 549,
   'geocodes': {'main': {'latitude': 34.03133, 'longitude': -118.462657},
    'roof': {'latitude': 34.03133, 'longitude': -118.462657}},
   'link': '/v3/places/56fea33a498e4c6f24e1bf4b',
   'location': {'address': '1814 Berkeley St',
    'census_block': '060377018011019',
    'country': 'US',
    'cross_street': 'Nebraska',
    'dma': 'Los Angeles',
    'formatted_address': '1814 Berkeley St (Nebraska), Santa Monica, CA 90404',
    'locality': 'Santa Monica',
    'postcode': '90404',
    'region': 'CA'},
   'name': 'Bondi Harvest',
   'related_places': {},
   'timezone': 'America/Los_Angeles'},
  {'fsq_id': '4a8875b7f964a520830620

In [76]:
vegan_la = []
for venue in vegan_restaurant_la["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    vegan_la.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

vegan_restaurant_df_la = pd.DataFrame(vegan_la)

vegan_restaurant_df_la

Name   Latitude   Longitude  Distance
0    Bondi Harvest  34.031330 -118.462657       549
1     Trader Joe's  34.026369 -118.455772       556
2          Habayit  34.030399 -118.448928       722
3  Viet Noodle Bar  34.026691 -118.456203       514
4       Sweetgreen  34.034839 -118.458764       435

In [78]:
# Creating a map centered around a location of vegan restaurant
m = folium.Map(location=[vegan_restaurant_df_la.iloc[0]['Latitude'], vegan_restaurant_df_la.iloc[0]['Longitude']], zoom_start=12)

for index, row in vegan_restaurant_df_la.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], 
                  tooltip=row['Name'], 
                  icon=folium.Icon(color='green')).add_to(m)

company_location = [34.031276, -118.456806]
folium.Marker(company_location, 
              tooltip='hulu',
              icon=folium.Icon(color='red', icon='cloud')).add_to(m)

m

In [92]:
# checking for close Airport 

def requests_for_foursquare (query, lat, lon, radius=500, limit=1):
    lat = 34.031276
    lon = -118.456806 
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [100]:
airport_la = requests_for_foursquare ("International Airport", lat, lon, radius=50000, limit=5)
airport_la

{'results': [{'fsq_id': '439ec330f964a520102c1fe3',
   'categories': [{'id': 19031,
     'name': 'Airport',
     'short_name': 'Airport',
     'plural_name': 'Airports',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/airport_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 10798,
   'geocodes': {'main': {'latitude': 33.94352, 'longitude': -118.40842},
    'roof': {'latitude': 33.944697, 'longitude': -118.398107}},
   'link': '/v3/places/439ec330f964a520102c1fe3',
   'location': {'address': '1 World Way',
    'census_block': '060379800281018',
    'country': 'US',
    'cross_street': '',
    'dma': 'Los Angeles',
    'formatted_address': '1 World Way, Los Angeles, CA 90045',
    'locality': 'Los Angeles',
    'po_box': 'P.O. Box 92216',
    'postcode': '90045',
    'region': 'CA'},
   'name': 'Los Angeles International Airport (LAX)',
   'related_places': {'children': [{'fsq_id': '4f17cbabe4b0f1e7bab9e1c3',
     

In [101]:
airport_la_ = []
for venue in airport_la["results"]:
    name = venue["name"]
    location = venue.get("geocodes", {}).get("main", {})
    latitude = location.get("latitude", "N/A")
    longitude = location.get("longitude", "N/A")
    distance = venue.get("distance", "N/A")
    airport_la_.append({"Name": name, "Latitude": latitude, "Longitude": longitude, "Distance": distance})

airport_df_la = pd.DataFrame(airport_la_)

airport_df_la

Name  Latitude   Longitude  Distance
0    Los Angeles International Airport (LAX)  33.94352 -118.408420     10798
1  Tom Bradley International Terminal (TBIT)  33.94352 -118.408537     10686

In [103]:
# Creating a map centered around a location of vegan restaurant
m = folium.Map(location=[airport_df_la.iloc[0]['Latitude'], airport_df_la.iloc[0]['Longitude']], zoom_start=12)

for index, row in airport_df_la.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], 
                  tooltip=row['Name'], 
                  icon=folium.Icon(color='green')).add_to(m)

company_location = [34.031276, -118.456806]
folium.Marker(company_location, 
              tooltip='hulu',
              icon=folium.Icon(color='red', icon='cloud')).add_to(m)

m

In [107]:
# checking the location from the closest airport from Los Angeles - USA

latitude = 34.031276
longitude = -118.456806

geolocator = Nominatim(user_agent="airport_finder")
airport_coordinates = (33.9439667, -118.4109258) 
distance = geodesic((latitude, longitude), airport_coordinates).miles
print(f"The closest airport is Tom Bradley International, Los Angeles, which is approximately {distance:.2f} miles away.")

The closest airport is Tom Bradley International, Los Angeles, which is approximately 6.57 miles away.


In [ ]:
"""
While the aiport is closer to the office in Los Angeles, more criteria were met for the office in San Fracisco
Deciding to place the office iin San Francisco, taking Zynga's place!


"""